# Files needed
* generated images

# Necessary installations

In [3]:
!pip install torch-fidelity

# Imports

In [4]:
from torchmetrics.image.inception import InceptionScore

import torch
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import pickle
from tqdm import tqdm

In [5]:
#torch.manual_seed(0)
#np.random.seed(0)
torch.use_deterministic_algorithms(True)

# Read inputs

In [6]:
# CHANGE HERE
modelName = 'DALL-E_512'

#DALL-E
if modelName == 'DALL-E_1024' or modelName == 'DALL-E_512':
    input_image_path = '' # path to the DALL 1024 or DALL 512 image files
    image_files = os.listdir(input_image_path)

elif modelName == 'SDXL_1024':
    image_files = '' # path to the SDXL_1024 pkl file

elif modelName == 'SDXL_TURBO_1024':
    image_files = '' # path to the SDXL_TUBRO_1024 pkl file

elif modelName == 'FLUX_SCHNELL_1024':
    image_files = '' # path to the FLUX_SCHNELL_1024 pkl file

elif modelName == 'SDXL_512':
    image_files = '' # path to the SDXL_512 pkl file

elif modelName == 'SDXL_TURBO_512':
    image_files = '' # path to the SDXL_TURBO_512 pkl file

elif modelName == 'FLUX_SCHNELL_512':
    image_files = '' # path to the FLUX_SCHNELL_512 pkl file

# Compute Inception Score (single value for all images)

In [8]:
inceptionScore = []

# just to check if the value of inception score is consistent across computations
for i in range(5):
    pixelTensorList = []
    
    if modelName == 'DALL-E_512' or modelName == 'DALL-E_1024':
        for i in tqdm(range(len(image_files))):
            image_path = os.path.join(input_image_path, image_files[i])
            im = cv2.imread(image_path)
            if modelName == 'DALL-E_512':
                im = cv2.resize(im, (512, 512))
            image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            pixels = np.array(image)
            pixelTensorList.append(torch.from_numpy(pixels))
                    
    elif modelName == 'SDXL_512' or modelName == 'SDXL_1024':
        with open(image_files, 'rb') as f:
            images = pickle.load(f)
        for im in tqdm(images):
            #image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            pixels = torch.from_numpy(np.array(im))
            pixelTensorList.append(pixels)
    
    elif modelName == 'FLUX_SCHNELL_512' or modelName == 'FLUX_SCHNELL_1024':
        with open(image_files, 'rb') as f:
            images = pickle.load(f)
        for im in tqdm(images):
            #image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            pixels = np.array(im)
            pixelTensorList.append(torch.from_numpy(pixels))
            
    elif modelName == 'SDXL_TURBO_512' or modelName == 'SDXL_TURBO_1024':
        with open(image_files, 'rb') as f:
            images = pickle.load(f)
        for im in tqdm(images):
            #image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            pixels = np.array(im)
            #print(pixels.shape)
            pixelTensorList.append(torch.from_numpy(pixels))
    
    pixelTensor = torch.stack(pixelTensorList).permute(0, 3, 1, 2)
    print(pixelTensor.shape)

    torch.manual_seed(0) 

    inception = InceptionScore()
   
    print(modelName)
    inception.update(pixelTensor)
    inceptionScore.append(inception.compute())
    #break

100%|██████████| 200/200 [00:08<00:00, 23.49it/s]
/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


torch.Size([200, 3, 512, 512])
xxxxxxxxx
DALL-E_512


100%|██████████| 200/200 [00:04<00:00, 46.31it/s]


torch.Size([200, 3, 512, 512])
xxxxxxxxx
DALL-E_512


100%|██████████| 200/200 [00:04<00:00, 47.27it/s]


torch.Size([200, 3, 512, 512])
xxxxxxxxx
DALL-E_512


100%|██████████| 200/200 [00:04<00:00, 46.60it/s]


torch.Size([200, 3, 512, 512])
xxxxxxxxx
DALL-E_512


100%|██████████| 200/200 [00:04<00:00, 46.39it/s]


torch.Size([200, 3, 512, 512])
xxxxxxxxx
DALL-E_512


In [9]:
for i in inceptionScore:
    print('inceptionScore: ', i)
    print('coefficientOfVariation: ', i[1]/i[0])

inceptionScore:  (tensor(8.9759), tensor(1.2859))
coefficientOfVariation:  tensor(0.1433)
inceptionScore:  (tensor(8.9759), tensor(1.2859))
coefficientOfVariation:  tensor(0.1433)
inceptionScore:  (tensor(8.9759), tensor(1.2859))
coefficientOfVariation:  tensor(0.1433)
inceptionScore:  (tensor(8.9759), tensor(1.2859))
coefficientOfVariation:  tensor(0.1433)
inceptionScore:  (tensor(8.9759), tensor(1.2859))
coefficientOfVariation:  tensor(0.1433)


In [10]:
import pickle

inceptionScoreFile = '' # path to write the inception score in a file (this is optional as the computation of IS is quick)
with open(inceptionScoreFile+modelName+'.pkl', 'wb') as f:
    pickle.dump(inceptionScore, f)

In [ ]:
# to ensure reproducibility in the random generation 
# https://pytorch.org/docs/stable/notes/randomness.html

# function used to create subsets in the inception score computation
# https://pytorch.org/docs/stable/generated/torch.chunk.html

# inception score lightning AI implementation
# https://github.com/Lightning-AI/torchmetrics/blob/master/src/torchmetrics/image/inception.py#L34-L220